In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import autovarlambda
import time
from hyperopt import hp, rand, tpe, Trials, fmin
import pandas as pd
import yaml

In [2]:
import GPy
import GPyOpt
from numpy.random import seed

In [4]:
with open("varlam_gpyopt.yml", 'r') as stream:
    data_loaded = yaml.load(stream)
maxevals = data_loaded.get("maxevals")
eps_up = data_loaded.get("eps_up")
eps_down = data_loaded.get("eps_down")
eps=eps_down/2.0
if eps_up > eps_down:
    eps=eps_up/2.0
nlamvar = data_loaded.get("nlamvar")
nener = data_loaded.get("nener")
NEMX = nener
listweight = data_loaded.get("weight")
dummylen = len(listweight)
if dummylen < NEMX:
    print(" ELEMENTS MISSING IN WEIGHT (.yaml). MUST BE == NENER")
    sys.exit()
if dummylen > NEMX:
    print(" EXTRA ELEMENTS IN WEIGHT (.yaml). MUST BE == NENER")
    sys.exit()
weight = np.array(listweight)
cfgs = data_loaded.get("cfgs")

In [6]:
# Define the bounds of the function
bounds =[{'name': 'lam1', 'type': 'continuous', 'domain': (0.1,2)},
         {'name': 'lam2', 'type': 'continuous', 'domain': (0.1,2)},
         {'name': 'lam3', 'type': 'continuous', 'domain': (0.1,2)}]

In [46]:
type(bounds)

list

In [50]:
bounds=[]
igrid=0
for i in range(NVMX):
    alam=str(i+1)
    if igrid==0:
        bounds.append({'name': 'lam'+alam, 'type': 'continuous', 'domain': (lv[i]-eps_down,lv[i]+eps_up)})

In [51]:
bounds

[{'name': 'lam1', 'type': 'continuous', 'domain': (0.09999999999999998, 2.0)},
 {'name': 'lam2', 'type': 'continuous', 'domain': (0.09999999999999998, 2.0)},
 {'name': 'lam3', 'type': 'continuous', 'domain': (0.09999999999999998, 2.0)}]

In [7]:
# Define useful functions
def atom_name(nzion):
    nzion=abs(nzion)
    atomname=['0','H','He','Li','Be','B','C','N','O','F','Ne','Na','Mg']
    for i in range(len(atomname)):
         if nzion==i:
              chatom = atomname[i]
    return chatom
def pot_type(nzion):
    if nzion < 0:
        chtype="STO"
    else:
        chtype="TFDA"
    return chtype

# Define function for writing iteration number in a human-readable-way
def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000
    return '%.f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])


In [28]:
def error_relat(valexact,valcomp):
    if valexact == 0.0 or valcomp == 0.0:
        error=999
    error = abs((valexact-valcomp)/valexact)
    return error

In [29]:
# Loss function defined as
#     weighted sum of energies relative errors
def loss_wsumE2(enere,enerc,neex,weight):
    loss = 0.0
    for i in range(neex):
        diff = error_relat(enere[i],enerc[i])
        loss = loss + weight[i]*diff**2
    return loss

In [30]:
# Define function to be minimized
def var_lambda(x):
    lam0 = x[0,:]
    nlam0 = len(lam0)
    autovarlambda.run_as(lam0,nlam0)
    neex = autovarlambda.exactbck.ne
    enere = autovarlambda.eneroutbck.enere
    enerc = autovarlambda.eneroutbck.enerc
    loss = loss_wsumE2(enere,enerc,neex,weight)
    print(lam0,loss)
    return loss

In [43]:
# Determine file name string vector from fortran subroutine data
def def_filename():
    ncfg=autovarlambda.salgebbck.mxconf
    nzion=autovarlambda.salgebbck.nzion
    chatom=atom_name(nzion)
    chtype=pot_type(nzion)
    cfgs=str(ncfg)+"CFG"+chtype
    filename=chatom+cfgs+"_GP"+human_format(maxevals)
    return ncfg, filename

# Open .out file
def open_fout(filename,ncfg):
    fener = open(filename+".out","w")
    return fener

# Define and initialize variables
def init_var(NVMX,NEMX):
    lam0 = np.array(NVMX)
    nlam0 = NVMX
    enere = np.zeros(NEMX)
    enerc = np.zeros(NEMX)
    neex = int()
    return lam0, nlam0, enere, enerc, neex

In [44]:
def print_eresults():
    lam_best = myBopt.x_opt
    loss_best = var_lambda(lam_best.reshape(-1,3))
    #print("x*= {} f(x*)= {}".format(myBopt.x_opt,myBopt.fx_opt))
    print(enere)
    print(enerc)
    erroregr = error_relat(enere[0],enerc[0])
    print("\n===>>> print results in .out <<<===")
    print("\n Number of evaluations = {:10d}".format(maxevals),file=fener)
    print("                  Time = {:10.3f} minutes".format(total),file=fener)
    print("-"*80,file=fener)
    print(" Best results:",file=fener)
    print("-"*80,file=fener)
    print("\n  lambda = {} ".format(lam_best),file=fener)
    print("\n  Ground State Energy  = {:12.6f}".format(enerc[0]),
          "\n                  NIST = {:12.6f}".format(enere[0]),
          "\n                   Er% = {:12.4f} %".format(erroregr),file=fener)
    for i in range(1,neex):
        errorex = error_relat(enere[i],enerc_best[i])
        print("\n       {} Excit. Energy = {:12.6f}".format(i,enerc[i]),
              "\n                  NIST = {:12.6f}".format(enere[i]),
              "\n                   Er% = {:12.6f} %".format(errorex),file=fener)
    print("\n            Total loss = {:12.4f} %".format(loss_best),file=fener)
    fener.close()

In [45]:
# Initialize variables
lv = np.full(nlamvar, 1.)
NVMX=len(lv)
ntcfg = len(cfgs)
lam0, nlam0, enere, enerc, neex = init_var(NVMX,NEMX)

# Begin loop in "configurations"
for i in range(ntcfg):
    icfg = cfgs[i]
    print("\n===>>> RUN {:3d} CFGs <<<===\n".format(icfg))
    cpdas = "cp das_" + str(icfg) + "CFG das"
    os.system(cpdas)
# Run initialization f2py subroutines
    autovarlambda.open_files()
    autovarlambda.inp_das()
    autovarlambda.inp_exact()

# if "ne" of exactvalues.dat is != than nener of .yaml
#  => "nener" is considered and ne is dumped
    dummyne = autovarlambda.exactbck.ne.copy()
    if nener != dummyne:
        print(" ************ nener != neex          ************")
        print(" ************ forcing: neex == nener ************\n")
        autovarlambda.exactbck.ne = nener

# Initialize files and necessary arrays
    ncfg, filename = def_filename()
    if ncfg != icfg:
        print(" >>>> configuration mismatch !!!! ")
        sys.exit()
    fener = open_fout(filename,ncfg)

# Creates three identical objects that we will later use to compare the optimization strategies 
    print("===>>> RUN GPyOpt <<<===")
    t0 = time.time()
    myBopt = GPyOpt.methods.BayesianOptimization(f=var_lambda,
                                                 domain=bounds,
                                                 model_type = 'GP',
                                                 acquisition_type='EI',  
                                                 normalize_Y = True,
                                                 acquisition_weight = 20)
# runs the optimization for the three methods
    myBopt.run_optimization(maxevals,verbosity=False)
    myBopt.save_evaluations(filename+".dat")
    myBopt.save_models("model_"+filename+".dat")
    myBopt.save_report("report_"+filename+".dat")
    t1 = time.time()
    total = (t1-t0)/60.

# run autostructure for best lambda found
    print_eresults()


===>>> RUN  32 CFGs <<<===

 ************ nener != neex          ************
 ************ forcing: neex == nener ************

===>>> RUN GPyOpt <<<===
[1.95905178 0.1185135  0.66163987] 1.6704713390159227
[0.88238976 0.35053126 1.3497884 ] 1.8386206781259373
[0.83508631 1.64145549 0.47302685] 0.07407908190370643
[0.18169062 1.73232034 0.11588754] 197.20766508387436
[0.98760596 1.55089963 0.64091576] 0.12388631757533262
[1.41862596 0.255242   0.99510074] 4.848952217717212
[1.23816203 1.83107284 0.36066885] 0.38586823689177185
[1.58563735 0.74158234 0.41242116] 1.1610531566226672
[2.         0.76270802 0.97006746] 0.12251598662911291
[0.83508631 1.64145549 0.47302685] 0.07407908190370643

===>>> print results in Be32CFGSTO_GP4.out <<<===


In [ ]:
os.system("rm CONFIG.DAT TERMS olg ols tmp das")